In [1]:
!pip install -r requirements.txt

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
!pip install -e .

Obtaining file:///tf/notebooks/EPT/1_taxonomist
  Attempting uninstall: taxonomist
    Found existing installation: taxonomist 0.0.3
    Can't uninstall 'taxonomist'. No files were found to uninstall.
  Running setup.py develop for taxonomist
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
!python scripts/preprocessing/process_biodiscover.py \
    --csv_path ../EPT-DL.csv \
    --out_folder data/processed/ept

       Specimen ID Sample Name/Number  ...        Label_DNA read count_DNA
0            4_F12            4_F12_1  ...     Goera pilosa       215998.0
1            4_F12            4_F12_1  ...     Goera pilosa       215998.0
2            4_F12            4_F12_1  ...     Goera pilosa       215998.0
3            4_F12            4_F12_1  ...     Goera pilosa       215998.0
4            4_F12            4_F12_2  ...     Goera pilosa       215998.0
...            ...                ...  ...              ...            ...
143270      10_C11           10_C11_2  ...  Nemoura cinerea       314403.0
143271      10_C11           10_C11_2  ...  Nemoura cinerea       314403.0
143272      10_C11           10_C11_2  ...  Nemoura cinerea       314403.0
143273      10_C11           10_C11_2  ...  Nemoura cinerea       314403.0
143274      10_C11           10_C11_2  ...  Nemoura cinerea       314403.0

[143275 rows x 37 columns]
data/processed/ept-14/01_EPT_processed.csv


In [4]:
import pandas as pd
df_csv = pd.read_csv("../EPT-DL.csv")
print(f"Number of classes: {len(df_csv['Final_species_label'].unique())}")
for label in df_csv['Final_species_label'].unique():
    print(label)

SyntaxError: invalid syntax (<ipython-input-4-69e8de5ca926>, line 3)

In [ ]:
!python scripts/01_train_test_split.py \
    --csv_path 'data/processed/ept/01_EPT_processed.csv' \
    --target_col 'Final_species_label' \
    --group_col 'individual' \
    --n_splits 5 \
    --random_state 42 \
    --out_folder 'data/processed/ept' \
    --shuffle 'False'

In [ ]:
df = pd.read_csv("data/processed/ept/01_EPT_processed_5splits_Final_species_label.csv")
df.head()

In [ ]:
import numpy as np
import pandas as pd

def even_pick(s, n):
    if len(s) <= n:
        return s
    else:
        indices = np.linspace(0, len(s)-1, num=n, dtype=int)
        return s.iloc[indices]
    
csv_list=['data/processed/ept/01_EPT_processed_5splits_Final_species_label.csv']
for num_img in [10, 25, 50, 75, 100, 150, 200, 300, 400, 500]:
    outlist = []
    for individual in df['individual'].unique():
        outlist.append(even_pick(df.query("individual == @individual"), num_img))

    df_num_img = pd.concat(outlist)
    df_num_img.to_csv(f'data/processed/ept/01_EPT_processed_5splits_Final_species_label_{str(num_img)}.csv')
    csv_list.append(f'data/processed/ept/01_EPT_processed_5splits_Final_species_label_{str(num_img)}.csv')

In [ ]:
df_num_img.groupby("individual").count().sort_values("Image Path", ascending=False)

In [ ]:
!python scripts/preprocessing/test_dataset_config.py \
    --data_folder "../0_images/" \
    --dataset_config "conf/user_datasets.py" \
    --dataset_name "biodiscover" \
    --csv_path "data/processed/ept/01_EPT_processed_5splits_Final_species_label.csv" \
    --label "Final_species_label" \
    --fold 0

In [ ]:
import wandb
wandb.login() #581221dff0063a897a4eff9d285fa871a2c8e7f2

In [ ]:
# Only used for testing different number of images per specimen for all folds
for csv in csv_list:
    for lr in [0.001, 0.00001]:
        for fold in range(5):
            !python scripts/02_train.py \
                --data_folder "../0_images/" \
                --dataset_config "conf/user_datasets.py" \
                --dataset_name "biodiscover" \
                --csv_path {csv} \
                --label "Final_species_label" \
                --fold {fold} \
                --class_map "data/processed/class_map_location.txt" \
                --imsize 224 \
                --batch_size 256 \
                --aug 'trivialaugment' \
                --load_to_memory 'False' \
                --model 'efficientnet_b0' \
                --opt 'adamw' \
                --max_epochs 200 \
                --min_epochs 50 \
                --early_stopping 'True' \
                --early_stopping_patience 50 \
                --criterion 'cross-entropy' \
                --lr {lr} \
                --auto_lr 'False' \
                --log_dir 'ept-14-img_num_research' \
                --out_folder 'outputs' \
                --out_prefix f'08_ept-14_img{csv[-7:-4]}_lr{lr}_test_run' \
                --deterministic 'True' \
                --precision '16-mixed'

In [ ]:
# Only used for testing different number of images per specimen for all folds
for csv in csv_list:
    for lr in [0.005, 0.001, 0.0005, 0.0001, 0.00005, 0.00001, 0.000005, 0.000001]:
        for fold in range(5):
            !python scripts/02_train.py \
                --data_folder "../0_images/" \
                --dataset_config "conf/user_datasets.py" \
                --dataset_name "biodiscover" \
                --csv_path {csv} \
                --label "Final_species_label" \
                --fold {fold} \
                --class_map "data/processed/class_map_location.txt" \
                --imsize 224 \
                --batch_size 256 \
                --aug 'trivialaugment' \
                --load_to_memory 'False' \
                --model 'efficientnet_b0' \
                --opt 'adamw' \
                --max_epochs 100 \
                --min_epochs 0 \
                --early_stopping 'False' \
                --early_stopping_patience 50 \
                --criterion 'cross-entropy' \
                --lr {lr} \
                --auto_lr 'False' \
                --log_dir 'ept-14-img_num_research' \
                --out_folder 'outputs' \
                --out_prefix f'08_ept-14_img{csv[-7:-4]}_lr{lr}_test_run' \
                --deterministic 'True' \
                --precision '16-mixed'